In [ ]:
import json
import networkx as nx
import meshplot as mp
import numpy as np
import open3d as o3d
import matplotlib.cm as cm

In [ ]:
MESH = 'faust_54098'
RAIZ = 50 #39

In [ ]:
def readCodeGraph( path ):
    graph = nx.DiGraph()
    with open( path ) as jsonFile:
        skel = json.load(jsonFile)
        root = skel['root']
        scale = skel['scale']
        graph.add_nodes_from( [ (joint['id'], {'code': np.asarray(joint['mean']), 'base':np.asarray(joint['base']), 'position':np.asarray(joint['position'])}) for joint in skel['joints'] ] )

        for joint in skel['joints']:
            graph.add_edges_from( [ (joint['id'], child) for child in joint['children']] )

    return graph, root, scale

In [ ]:
graphOfJoints, root, scale = readCodeGraph(f'../../results/{MESH}/{MESH}.json')

In [ ]:
def hijos( graph, nodo ):
    for hijo in nx.dfs_tree(graph, nodo):
        yield hijo

In [ ]:

#p = mp.plot( np.array( [np.asarray( graphOfJoints.nodes[node]['position']) for node in graphOfJoints.nodes]), return_plot = True, shading={'point_size': 0.1} )
#p.add_points(np.array([np.asarray( graphOfJoints.nodes[i]['position']) for i in hijos(graphOfJoints, RAIZ)]), shading={'point_color': 'blue', 'point_size':0.15} )

#for u,v in graphOfJoints.edges:
#    p.add_lines( np.asarray(graphOfJoints.nodes[u]['position']), np.asarray(graphOfJoints.nodes[v]['position']))

In [ ]:
# roto el brazo con respecto a articulacion
#angulo = -3* np.pi / 4
#rotationMatrix = np.array([
#    [1, 0 , 0],
#    [0, np.cos(angulo), -np.sin(angulo)],
#    [0, np.sin(angulo), np.cos(angulo)]
#])
#limbRootPosition = np.asarray(graphOfJoints.nodes[RAIZ]['position'])
#for nodo in hijos(graphOfJoints, RAIZ):
#    posicion = np.asarray(graphOfJoints.nodes[nodo]['position'])
#    nuevaPos = rotationMatrix @ (posicion - limbRootPosition) + limbRootPosition
#
#    graphOfJoints.nodes[nodo]['position'] = nuevaPos
#    graphOfJoints.nodes[nodo]['base'] = rotationMatrix @ graphOfJoints.nodes[nodo]['base']
#
#p = mp.plot( np.array( [np.asarray( graphOfJoints.nodes[node]['position']) for node in graphOfJoints.nodes]), return_plot = True, shading={'point_size': 0.1} )
#p.add_points(np.array([np.asarray( graphOfJoints.nodes[i]['position']) for i in hijos(graphOfJoints, RAIZ)]), shading={'point_color': 'blue', 'point_size':0.15} )

In [ ]:
def rotateLimb( pathFrom, pathTo, graph, limbRoot, angle ):

    class NumpyEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, np.ndarray):
                return obj.tolist()
            elif isinstance(obj, np.float32):
                return obj.tolist()
            return json.JSONEncoder.default(self, obj)
    
    with open( pathFrom, 'r' ) as jsonFile:
        data = json.load(jsonFile)
    
    limbNodes= list(hijos(graph, limbRoot))
    rotationMatrix = np.array([
        [1, 0 , 0],
        [0, np.cos(angle), -np.sin(angle)],
        [0, np.sin(angle), np.cos(angle)]
    ])
    limbRootPosition = np.asarray(graph.nodes[limbRoot]['position'])
    for joint in data['joints']:
        if joint['id'] in limbNodes:
            joint['position'] = rotationMatrix @ (np.asarray(joint['position']) - limbRootPosition) + limbRootPosition
            joint['base'] = rotationMatrix @ np.asarray(joint['base'])

    with open( pathTo, 'w+') as jsonFile:
        json.dump( data, jsonFile, cls=NumpyEncoder )

In [ ]:
rotateLimb( f'../../results/{MESH}/{MESH}.json', f'../../results/{MESH}/{MESH}_t.json', graph=graphOfJoints, limbRoot=RAIZ, angle=-3* np.pi / 4 )

In [ ]:
meshes_gt = []
with open(f'../../results/{MESH}/{MESH}_t.json') as jsonFile:
    skel = json.load(jsonFile)
    
    for joint in skel['joints']:
        T = (np.block( [[np.eye(3,3), np.asarray(joint['position']).reshape((3,1))],[np.eye(1,4,k=3)] ] ) @ 
             np.block( [ [np.asarray(joint['base']), np.zeros((3,1))], [np.eye(1,4,k=3)]]) @ 
             np.diag( [1 / skel['scale']] * 3 + [1] ))
        mesh = o3d.geometry.TriangleMesh( o3d.utility.Vector3dVector(np.array(joint['vertices']).astype(np.float32)), o3d.utility.Vector3iVector(np.array(joint['triangles']).astype(np.float32)) )
        mesh.transform( T )
        meshes_gt.append(mesh)


In [ ]:

p = None
cmap = cm.get_cmap('flag')
for i, mesh in enumerate(meshes_gt):
    if i < 70:
        if p is None:
            p = mp.plot( np.asarray(mesh.vertices), np.asarray(mesh.triangles), return_plot = True, c=np.array(cmap((i)/len(meshes_gt)))[0:3])
        else:
            p.add_mesh( np.asarray(mesh.vertices), np.asarray(mesh.triangles), c=np.array(cmap((i)/len(meshes_gt)))[0:3] )